In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import numpy as np
import copy

# Settings for printing numpy arrays
np.set_printoptions(precision=4, edgeitems=50, suppress=True)
np.core.arrayprint._line_width = 250

# Simplex Method in tableau format

Below is defined a function which comprises the Simplex Method under the table solution approach to solve Linear Programming Problems where we have to minimize or maximize an objective function under some restrictions.

In [3]:
def simplex_tableau(tableau, variables, basic_indexes, non_basic_indexes, itera = 1):
    
    basic_variables = [variables[i] for i in basic_indexes]
    non_basic_variables = [variables[i] for i in non_basic_indexes]
        
    # Printing our initial parameters for this iteration
    print("\n\n\033[1mIteration ", itera, "\033[0m")
    print("[\033[94m x_B \033[0m, \033[91m x_N \033[0m] = [ \033[94m"
          + ', '.join(basic_variables) + "\033[0m, \033[91m" + ', '.join(non_basic_variables) + "\033[0m ]\n")
    
    # Extracting elements from the tableau of the current iteration
    m = np.shape(tableau)[0] - 1
    n_plus_m = np.shape(tableau)[1] - 2
    ZXN = np.ravel(tableau[0, [i + 1 for i in non_basic_indexes]])
    XBXN = tableau[1:, [i + 1 for i in non_basic_indexes]]
    ZRHS = np.array([tableau[0, n_plus_m + 1]])
    XBRHS = (tableau[1:, n_plus_m + 1]).reshape(1, m)
    b_ = np.concatenate((np.ravel(XBRHS), np.zeros(n_plus_m - m)))
    
    print("\nEntry tableau")
    print(tableau)
    
    z_minus_c_max = max(ZXN)
    k = np.argmax(ZXN)
    
    # Validating if we stop or not the optimization
    if z_minus_c_max <= 0:   # We stop
        
        # Printing the final results
        print("\n\n\033[1mOptimality reached\033[0m")
        print("\nThe optimal BFS is")
        results_ls = [str(round(i, 4)) for i in b_]
        print("\033[92m[ " + ', '.join(basic_variables + non_basic_variables) + " ] = [ " + ', '.join(results_ls) + " ]\033[0m")
        perf_z = ZRHS[0]
        print("\nWith performance z =", perf_z, "\n\n")
        
        indexes_variables = [int(var[1:]) for var in basic_variables + non_basic_variables]
        dict_solution = dict(zip(indexes_variables, [b_i for b_i in b_]))
        sorted_dict_solution = dict(sorted(dict_solution.items(), key = lambda x:x[0]))
        solution = np.array(list(sorted_dict_solution.values()))
        #print(solution)
        return solution
        
    else:   # We continue
        
        # Calculating and printing the y_ki's
        y_k = np.ravel(XBXN[:, k])
        
        print("\nk =", k + 1, "-> column no.", k + 1, "of x_N part (", non_basic_variables[k], ")")
        print("y_k")
        print(y_k)
        
        # Analyzing if the optimal BFS is or not boundable by the condition y_k > 0
        flag = True
        aux_counter = 0
        for y_i in y_k:
            if y_i > 0:
                aux_counter = aux_counter + 1      
        if aux_counter == 0:
            flag = False

        if flag:   # Boundable
            
            # Calculating and printing x_k by the minimum quotient (current BFS divided by the y_k > 0) with its index r
            quot_ls = list()
            index_r_ls = list()
            for i in range(m):
                if y_k[i] > 0:
                    quot_ls.append(b_[i] / y_k[i])
                    index_r_ls.append(i)
            quot_arr = np.array(quot_ls)
            index_r_arr = np.array(index_r_ls)
            r = index_r_arr[np.argmin(quot_arr)]
            x_Br = min(quot_arr)
            print("\nr =", r + 1, "-> column no.", r + 1, "of x_B part (", basic_variables[r], ")")
            print("x_Br")
            print(x_Br)
            
            # Pivoting
            pivot = y_k[r]
            print("\npivot =", pivot)
            
            tableau[r + 1, :] = tableau[r + 1, :] / pivot
            for i in range(m + 1):
                if i != r + 1:
                    tableau[i, :] = tableau[i, :] - tableau[i, non_basic_indexes[k] + 1] * tableau[r + 1, :]
            
            print("\nPivoted tableau")
            print(tableau)
            # Exchanging the indexes
            print("\n\033[94m", non_basic_variables[k], "enters\033[0m and \033[91m", basic_variables[r], "leaves\033[0m the basis")
            aux_vars = basic_indexes[r]
            basic_indexes[r] = non_basic_indexes[k]
            non_basic_indexes[k] = aux_vars
            # Exchanging columns in tableau
            #aux_column = tableau[:, m + k + 1]
            #tableau[:, m + k + 1] =  tableau[:, 1 + r]
            #tableau[:, 1 + r] = aux_column
            
            # Recursive call
            return simplex_tableau(tableau, variables, basic_indexes, non_basic_indexes, itera + 1)

        else:   # Not boundable
            
            print("\n\n\033[1mOptimization process stopped :(\033[0m")
            print("\nThe optimal BFS is not boundable\n\n")
            
            return False
            

In [4]:
def construct_tableau(A, b, c, basic_indexes):
    m, n_plus_m = np.shape(A)
    B = A[:m, basic_indexes]
    c_b = c[basic_indexes]
    b_ = np.linalg.inv(B) @ b

    one_array = np.ones(1, dtype=int)
    zero_array_v = np.ravel(np.zeros(m, dtype=int)).reshape(1, m)
    ZX = np.ravel((c_b @ np.linalg.inv(B) @ A) - c)
    XBX = np.linalg.inv(B) @ A
    ZRHS = np.full((1), c_b @ np.ravel(b_))
    XBRHS = b_.reshape(1, m)

    tableau = np.vstack((np.concatenate((one_array, ZX, ZRHS), axis = 0).reshape(1, n_plus_m + 2),
                  np.concatenate((zero_array_v.T, XBX, XBRHS.T), axis = 1)))
    
    return tableau

This approach of the Simplex Method needs to preprocess our initial data in order to adjust it into a a particular tableau with the below entries:

|         | $z$ | $x_{B}$ | $x_{N}$                | $RHS$         |
|:---:|:---:|:---:|:---:|:---:|
| $z$     | $1$ | $0$     | $c_{b}B^{-1}N - c_{N}$ | $c_b \bar{b}$ |
| $X_{B}$ | $0$ | $I$     | $B^{-1}N$              | $\bar{b}$     |

That if we break it down we get the following equivalent tableau

|          | $z$      | $x_1$      | $x_2$      | $...$ | $x_n$      | $x_{n+1}$         | $...$ | $x_{n+m}$         | $RHS$         |
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
| $z$      | $1$      | $z_1-c_1$  | $z_2-c_2$  | $...$ | $z_n-c_n$  | $z_{n+1}-c_{n+1}$ | $...$ | $z_{n+m}-c_{n+m}$ | $c_b \bar{b}$ |
| $X_{B1}$ | $0$      | $Y_{1, 1}$ | $Y_{1, 2}$ | $...$ | $Y_{1, n}$ | $Y_{1, n+1}$      | $...$ | $Y_{1, n+m}$      | $\bar{b_1}$        |
| $X_{B2}$ | $0$      | $Y_{2, 1}$ | $Y_{2, 2}$ | $...$ | $Y_{2, n}$ | $Y_{2, n+1}$      | $...$ | $Y_{2, n+m}$      | $\bar{b_2}$          |
| $\vdots$ | $\vdots$ | $\vdots$   | $\vdots$   |       | $\vdots$   | $\vdots$          |       | $\vdots$          | $\vdots$      |
| $X_{Bm}$ | $0$      | $Y_{m, 1}$ | $Y_{m, 2}$ | $...$ | $Y_{m, n}$ | $Y_{m, n+1}$      | $...$ | $Y_{m, n+m}$      | $\bar{b_m}$          |

So, once we have already defined the previous function, we are able to apply it.

#### Example 1:
Solve the next linear programming problem:

$$min \hspace{0.5cm} x_1 + x_2 -4x_3$$
$$under \hspace{0.5cm} x_1 + x_2 + 2x_3 \leq 9$$
$$\hspace{1.4cm} x_1 + x_2 - x_3 \leq 2$$
$$\hspace{1.4cm} -x_1 + x_2 + x_3 \leq 4$$
$$\hspace{1cm}x_1, x_2, x_3 \geq 0$$

First, we introduce our lack variables $x_4, x_5, x_6$ and the problem can be rewritten as
$$min \hspace{0.5cm} x_1 + x_2 -4x_3 + 0x_4 + 0x_5 + 0x_6$$
$$under \hspace{0.5cm} x_1 + x_2 + 2x_3 + x_4 + 0x_5 + 0x_6 = 9$$
$$\hspace{1.4cm} x_1 + x_2 - x_3 + 0x_4 + x_5 + 0x_6 = 2$$
$$\hspace{1.4cm} -x_1 + x_2 + x_3 + 0x_4 + 0x_5 + x_6 = 4$$
$$\hspace{1cm}x_1, x_2, x_3, x_4, x_5, x_6 \geq 0$$

Then, initially considering $x_4, x_5, x_6$ as the basic variables and $x_1, x_2, x_3$ as the non basic ones, we set our parameters that will enter to the method as presented below:
\begin{equation} A = [N | B] = 
\left[
\begin{array}{ccc | ccc}
1 & 1 & 2 & 1 & 0 & 0\\
1 & 1 & -1 & 0 & 1 & 0\\
-1 & 1 & 1 & 0 & 0 & 1\\
\end{array}
\right]
\end{equation}

\begin{equation} b = 
\left[
\begin{array}{r}
9\\
2\\
4\\
\end{array}
\right]
\end{equation}


\begin{equation} c = [c_{B} | c_{N}] =
\left[
\begin{array}{ccc|ccc}
1 & 1 & -4 & 0 & 0 & 0\\
\end{array}
\right]
\end{equation}

In [5]:
A = np.array([[1, 1, 2, 1, 0, 0], [1, 1, -1, 0, 1, 0], [-1, 1, 1, 0, 0, 1]])
b = np.array([9, 2, 4])
c = np.array([1, 1, -4, 0, 0, 0])
variables = ['x' + str(i) for i in range(1, len(c) + 1)]

In [6]:
basic_indexes = [3, 4, 5]
non_basic_indexes = [0, 1, 2]

In [7]:
tableau = construct_tableau(A, b, c, basic_indexes)

In [8]:
tableau

array([[ 1., -1., -1.,  4.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  1.,  2.,  1.,  0.,  0.,  9.],
       [ 0.,  1.,  1., -1.,  0.,  1.,  0.,  2.],
       [ 0., -1.,  1.,  1.,  0.,  0.,  1.,  4.]])

In [9]:
simplex_tableau(copy.deepcopy(tableau), copy.deepcopy(variables), copy.deepcopy(basic_indexes), copy.deepcopy(non_basic_indexes))



Iteration  1 
[ x_B ,  x_N ] = [ x4, x5, x6, x1, x2, x3 ]


Entry tableau
[[ 1. -1. -1.  4.  0.  0.  0.  0.]
 [ 0.  1.  1.  2.  1.  0.  0.  9.]
 [ 0.  1.  1. -1.  0.  1.  0.  2.]
 [ 0. -1.  1.  1.  0.  0.  1.  4.]]

k = 3 -> column no. 3 of x_N part ( x3 )
y_k
[ 2. -1.  1.]

r = 3 -> column no. 3 of x_B part ( x6 )
x_Br
4.0

pivot = 1.0

Pivoted tableau
[[  1.   3.  -5.   0.   0.   0.  -4. -16.]
 [  0.   3.  -1.   0.   1.   0.  -2.   1.]
 [  0.   0.   2.   0.   0.   1.   1.   6.]
 [  0.  -1.   1.   1.   0.   0.   1.   4.]]

 x3 enters and  x6 leaves the basis


Iteration  2 
[ x_B ,  x_N ] = [ x4, x5, x3, x1, x2, x6 ]


Entry tableau
[[  1.   3.  -5.   0.   0.   0.  -4. -16.]
 [  0.   3.  -1.   0.   1.   0.  -2.   1.]
 [  0.   0.   2.   0.   0.   1.   1.   6.]
 [  0.  -1.   1.   1.   0.   0.   1.   4.]]

k = 1 -> column no. 1 of x_N part ( x1 )
y_k
[ 3.  0. -1.]

r = 1 -> column no. 1 of x_B part ( x4 )
x_Br
0.3333333333333333

pivot = 3.0

Pivoted tableau
[[  1.       0.      -4.   

array([0.3333, 0.    , 4.3333, 0.    , 6.    , 0.    ])

#### Example 2:
Solve the next linear programming problem:

$$max \hspace{0.5cm} 3x_1 + 2x_2$$
$$under \hspace{0.5cm} 2x_1 - 3x_2 \leq 3$$
$$\hspace{1cm}-x_1 + x_2 \leq 5$$
$$\hspace{1cm}x_1, x_2 \geq 0$$

First, we introduce our lack variables $x_3, x_4$ and then multiply by $-1$ the objective function in order to turn the maximization problem into a minimization one. So the problem can be rewritten as
$$min \hspace{0.5cm} -3x_1 -2x_2 + 0x_3 + 0x_4$$
$$under \hspace{0.5cm} 2x_1 - 3x_2 + x_3 + 0x_4 = 3$$
$$\hspace{1cm}-x_1 + x_2 + 0x_3 + x_4 = 5$$
$$\hspace{1cm}x_1, x_2, x_3, x_4 \geq 0$$

Then, initially considering $x_3, x_4$ as the basic variables and $x_1, x_2$ as the non basic ones, we set our parameters that will enter to the method as presented below:
\begin{equation} A = [N | B] = 
\left[
\begin{array}{rr|rr}
2 & -3 & 1 & 0\\
-1 & 1 & 0 & 1\\
\end{array}
\right]
\end{equation}

\begin{equation} b = 
\left[
\begin{array}{r}
3\\
5\\
\end{array}
\right]
\end{equation}


\begin{equation} c = [c_{B} | c_{N}] =
\left[
\begin{array}{rr|rr}
-3 & -2 & 0 & 0\\
\end{array}
\right]
\end{equation}

In [10]:
A = np.matrix([[2, -3, 1, 0], [-1, 1, 0, 1]])
b = np.array([3, 5])
c = np.array([-3, -2, 0, 0])
variables = ["x1", "x2", "x3", "x4"]

In [11]:
basic_indexes = [2, 3]
non_basic_indexes = [0, 1]

In [12]:
tableau = construct_tableau(A, b, c, basic_indexes)

In [13]:
tableau

matrix([[ 1.,  3.,  2.,  0.,  0.,  0.],
        [ 0.,  2., -3.,  1.,  0.,  3.],
        [ 0., -1.,  1.,  0.,  1.,  5.]])

In [14]:
simplex_tableau(copy.deepcopy(tableau), copy.deepcopy(variables), copy.deepcopy(basic_indexes), copy.deepcopy(non_basic_indexes))



Iteration  1 
[ x_B ,  x_N ] = [ x3, x4, x1, x2 ]


Entry tableau
[[ 1.  3.  2.  0.  0.  0.]
 [ 0.  2. -3.  1.  0.  3.]
 [ 0. -1.  1.  0.  1.  5.]]

k = 1 -> column no. 1 of x_N part ( x1 )
y_k
[ 2. -1.]

r = 1 -> column no. 1 of x_B part ( x3 )
x_Br
1.5

pivot = 2.0

Pivoted tableau
[[ 1.   0.   6.5 -1.5  0.  -4.5]
 [ 0.   1.  -1.5  0.5  0.   1.5]
 [ 0.   0.  -0.5  0.5  1.   6.5]]

 x1 enters and  x3 leaves the basis


Iteration  2 
[ x_B ,  x_N ] = [ x1, x4, x3, x2 ]


Entry tableau
[[ 1.   0.   6.5 -1.5  0.  -4.5]
 [ 0.   1.  -1.5  0.5  0.   1.5]
 [ 0.   0.  -0.5  0.5  1.   6.5]]

k = 2 -> column no. 2 of x_N part ( x2 )
y_k
[-1.5 -0.5]


Optimization process stopped :(

The optimal BFS is not boundable




False